Resources used while writing this notebook:
- [Nimish Sanghi's book on Deep RL](https://www.amazon.com/Deep-Reinforcement-Learning-Python-TensorFlow/dp/1484268083)
- Tawsif Kamal's videos on Blackjack Monte Carlo Reinforcement Learning [part 1](https://youtu.be/NeusGkowXR4?si=9a1aE_bInK4vSAHw) and [part 2](https://youtu.be/wn8hlPNwL74?si=PV_h3WQCXmZKwRzW).
- [Gymnasium docs for Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/).

In [ ]:
class GLIE_MC_Control:
    def __init__(self):
        pass

    def every_visit(self):
        """
        This is the every-visit GLIE Monte Carlo Control algorithm.
        Check pg. 88 of Nimish Sanghi's book for the pseudocode.
        """
        pass

    def first_visit(self):
        """
        This is the first-visit GLIE Monte Carlo Control algorithm.
        """
        pass